In [1]:
import sklearn
import numpy as np
import scipy
import csv

def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix(( loader['data'], loader['indices'], loader['indptr']),
                     shape = loader['shape'])
        
def load_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = '\n')
        array = [float(row[0]) for row in reader]
        return array

In [2]:
data_train = load_sparse_csr('DB_no_cleaning/data_train_no_feat.npz')
data_test = load_sparse_csr('DB_no_cleaning/data_test_no_feat.npz')

In [3]:
label_train = load_csv('DB_no_cleaning/label_train_no_feat.csv')
label_test = load_csv('DB_no_cleaning/label_test_no_feat.csv')

In [4]:
def score(true_label, predicted_label):
    length = len(true_label)
    total = 0
    for i, label in enumerate(true_label):
        if label == predicted_label[i]:
            total += 1
    return float(total)/float(length)

In [5]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

# n_estimators is the number of trees we want to make (higher is generally better)
# min_samples_split is tfnhe minimum number of rows we need to make a split
# min_samples_leaf is the minimum number of samples we can have at the place where a tree branch ends (the bottom points of the tree)
alg = RandomForestClassifier(random_state=1, n_estimators=30, min_samples_split=2, min_samples_leaf=1)
alg.fit(data_train, label_train)
predicted_label = alg.predict(data_test)


#scores = cross_validation.cross_val_score(alg, data_train, label_train, cv=5)
#print("Random Forest - Mean score on cross val", scores.mean())
print("Random Forest - Score on test_data : ", score(label_test, predicted_label))

('Random Forest - Score on test_data : ', 0.81248)


In [7]:
"""import matplotlib.pyplot as plt
alg.fit(data_train, label_train)
#Plot feature importance
feature_importance = alg.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos)
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()"""

"import matplotlib.pyplot as plt\nalg.fit(data_train, label_train)\n#Plot feature importance\nfeature_importance = alg.feature_importances_\n# make importances relative to max importance\nfeature_importance = 100.0 * (feature_importance / feature_importance.max())\nsorted_idx = np.argsort(feature_importance)\npos = np.arange(sorted_idx.shape[0]) + .5\nplt.subplot(1, 2, 2)\nplt.barh(pos, feature_importance[sorted_idx], align='center')\nplt.yticks(pos)\nplt.xlabel('Relative Importance')\nplt.title('Variable Importance')\nplt.show()"

In [8]:
alg.fit(data_train, label_train)
alg.feature_importances_

/home/bat/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:267: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [9]:
#[i  for i in range(len(alg.feature_importances_)) if alg.feature_importances_[i]>0.01]

In [10]:
feature_importances = alg.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]
print alg.feature_importances_[sorted_idx[0:10]]

[ 0.00799602  0.00678455  0.00393853  0.0031856   0.002951    0.00281542
  0.0026979   0.00264985  0.00207746  0.00202144]


In [11]:
nb_best_features = 100
best_features = alg.feature_importances_[sorted_idx[0:nb_best_features]]

In [12]:
feature_names = open("feature_names.csv")
feature_name = [line for line in feature_names]
print [feature_name[i] for i in sorted_idx[0:nb_best_features]]

['worst\n', 'bad\n', 'waste\n', 'inter\n', 'stupid\n', 'great\n', 'terrible\n', 'worse\n', 'love\n', 'even\n', 'awful\n', 'supposed\n', 'boring\n', 'best\n', 'crap\n', 'amazing\n', 'plot\n', 'one best\n', 'movie\n', 'nothing\n', 'ridiculous\n', 'excellent\n', 'could\n', 'don\n', 'wonderful\n', 'minutes\n', 'would\n', 'fantastic\n', 'avoid\n', 'like\n', 'horrible\n', 'that\n', 'susp\n', 'waste time\n', 'least\n', 'dull\n', 'well\n', 'also\n', 'didn\n', 'rev_length\n', 'it\n', 'there\n', 'young\n', 'mess\n', 'what\n', 'save\n', 'one worst\n', 'no\n', 'poorly\n', 'money\n', 'badly\n', 'acting\n', 'laughable\n', 'watching\n', '10 10\n', 'instead\n', 'lame\n', 'better\n', 'something\n', 'unfortunately\n', 'cheap\n', 'perfect\n', 'make\n', 'highly\n', 'budget\n', 'story\n', 'idea\n', 'today\n', 'performances\n', 'the\n', 'fun\n', 'unless\n', 'good\n', 'really bad\n', 'script\n', 'wasn\n', 'always\n', 'still\n', 'the acting\n', 'one\n', 'enjoy\n', 'decent\n', 'poor\n', 'predictable\n', 'doesn

In [15]:
import csv

def save_sparse_csr(filename, array):
    np.savez(filename, data = array.data, indices = array.indices, 
             indptr = array.indptr, shape = array.shape)

In [19]:
save_sparse_csr ("data_train_best",data_train[:,sorted_idx[0:20000]])
save_sparse_csr ("data_test_best",data_test[:,sorted_idx[0:20000]])

In [46]:
feature_importances[sorted_idx]

1.0000000000000002

In [47]:

percentage=0.8
perc_var = 0
selected_features = []
for idx, i in enumerate(sorted_idx):
    if(idx%500 ==0):
        print idx, perc_var
    perc_var+= feature_importances[i] #pas super precis donc en les sommant tous comme ça on arrive a moins de 1.0
    selected_features.append(feature_name[i])
    if(perc_var> percentage):
        break

0 0
100 0.134542040572
200 0.189738505709
300 0.230161031742
400 0.261708347002
500 0.288270163265
600 0.311308750446
700 0.331840461978
800 0.350312396179
900 0.367046075459
1000 0.382303122306
1100 0.396295845517
1200 0.409136000126
1300 0.420813359649
1400 0.431645706558
1500 0.441722202502
1600 0.451091606822
1700 0.459901787041
1800 0.468191771168
1900 0.475967239681
2000 0.483271153201
2100 0.490153801296
2200 0.49663834893
2300 0.502794184792
2400 0.508708947386
2500 0.514344051099
2600 0.519710982194
2700 0.524853299535
2800 0.529799357933
2900 0.534573412002
3000 0.539170473959
3100 0.543625799713
3200 0.547940945842
3300 0.552093808987
3400 0.556119181112
3500 0.56002361807
3600 0.563806837064
3700 0.567481384772
3800 0.571056538353
3900 0.574545868027
4000 0.577949492542
4100 0.581255744365
4200 0.584490958424
4300 0.587648951244
4400 0.590737322703
4500 0.593765398051
4600 0.596724927831
4700 0.599616550938
4800 0.602448615548
4900 0.605216246532
5000 0.607930436724
5100 0.

In [13]:
import matplotlib.pyplot as plt
pos = np.arange(best_features.shape[0]) + .5
plt.barh(pos, best_features, align='center')
plt.yticks(pos, sorted_idx[0:nb_best_features])
plt.xlabel('Importance')
plt.show()